### Download the Data
http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=ba8f8e41c6ae0b614cb01e70e1fbc1015814ba09d3291947234d49d15c2e746c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import wget
fn = wget.download('http://files.grouplens.org/datasets/movielens/ml-latest-small.zip')
fn

'ml-latest-small.zip'

In [3]:
!unzip ml-latest*

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


### Basic Info About Data

In [0]:
PATH = 'ml-latest-small'

In [5]:
!find $PATH -name '*.csv' | xargs wc -l | sort -nr

 124007 total
 100837 ml-latest-small/ratings.csv
   9743 ml-latest-small/movies.csv
   9743 ml-latest-small/links.csv
   3684 ml-latest-small/tags.csv


In [6]:
# find which seperator is used to seperate the columns of each csv file
!head -1 $PATH/*.csv


==> ml-latest-small/links.csv <==
movieId,imdbId,tmdbId

==> ml-latest-small/movies.csv <==
movieId,title,genres

==> ml-latest-small/ratings.csv <==
userId,movieId,rating,timestamp

==> ml-latest-small/tags.csv <==
userId,movieId,tag,timestamp


> * We can see that all files are comma-seperated.

### Read the Dataset

In [0]:
import pandas as pd
import numpy as np
import re
import os

from collections import defaultdict

In [0]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 1000)

In [0]:
db = defaultdict(pd.DataFrame)

for filename in os.listdir(PATH):
    if filename.endswith('.csv'):
        db[filename.split('.')[0]] = pd.read_csv(f"{PATH}/{filename}")
        

In [10]:
pd.DataFrame(db.items(), columns=['table_name', 'table_data'])

,table_name,table_data
0,links,movieId imdbId tmdbId 0 1 114709 862.0 1 2 113497 8844.0 2 3 113228 15602.0 3 4 114885 31357.0 4 5 113041 11862.0 ... ... ... ... 9737 193581 5476944 432131.0 9738 193583 5914996 445030.0 9739 193585 6397426 479308.0 9740 193587 8391976 483455.0 9741 193609 101726 37891.0 [9742 rows x 3 columns]
1,ratings,userId movieId rating timestamp 0 1 1 4.0 964982703 1 1 3 4.0 964981247 2 1 6 4.0 964982224 3 1 47 5.0 964983815 4 1 50 5.0 964982931 ... ... ... ... ... 100831 610 166534 4.0 1493848402 100832 610 168248 5.0 1493850091 100833 610 168250 5.0 1494273047 100834 610 168252 5.0 1493846352 100835 610 170875 3.0 1493846415 [100836 rows x 4 columns]
2,tags,userId movieId tag timestamp 0 2 60756 funny 1445714994 1 2 60756 Highly quotable 1445714996 2 2 60756 will ferrell 1445714992 3 2 89774 Boxing story 1445715207 4 2 89774 MMA 1445715200 ... ... ... ... ... 3678 606 7382 for katie 1171234019 3679 606 7936 austere 1173392334 3680 610 3265 gun fu 1493843984 3681 610 3265 heroic bloodshed 1493843978 3682 610 168248 Heroic Bloodshed 1493844270 [3683 rows x 4 columns]
3,movies,movieId title genres 0 1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy 1 2 Jumanji (1995) Adventure|Children|Fantasy 2 3 Grumpier Old Men (1995) Comedy|Romance 3 4 Waiting to Exhale (1995) Comedy|Drama|Romance 4 5 Father of the Bride Part II (1995) Comedy ... ... ... ... 9737 193581 Black Butler: Book of the Atlantic (2017) Action|Animation|Comedy|Fantasy 9738 193583 No Game No Life: Zero (2017) Animation|Comedy|Fantasy 9739 193585 Flint (2017) ...


### Connect To Database Engine - 
> * We use sqlite database engine to perform sql query
> * There are many packages to connect to sqlite engine such as - sqlite3, sqlalchemy etc.
> * We use sqlalchemy to connect to database engine. It works with many Relational Database Management Systems.
> * We will use pandas DataFrame class to represent the result of a sql query into a table form.

In [0]:
from sqlalchemy import create_engine

In [0]:
engine = create_engine("sqlite:///movie_review.sqlite")

In [13]:
# print the names of table in the database movie_review.sqlite
table_names = engine.table_names()
pd.DataFrame([table_names])

""
0


> * Since there is no table in movie_review.sqlite database, we will create some tables and insert data of movie review into them.

> * https://sqlite.org/cli.html

In [0]:
# connect to database
con = engine.connect()

# create table one by one 
for table_name, table_data in db.items():    
    table_data.to_sql(table_name, con=engine, index=False,
                      if_exists='replace')        

In [15]:
# print the names of table in the database movie_review.sqlite
table_names = engine.table_names()
pd.DataFrame([table_names])

,0,1,2,3
0,links,movies,ratings,tags


### Query 1: Display Top/Last 5 rows of a table

> * **SQL** 

In [0]:
table_name = 'links'

In [17]:
# Top 5 rows
rs = con.execute(f'''SELECT * 
                     FROM {table_name}
                     LIMIT 5''')
pd.DataFrame(rs.fetchall(), columns=db[table_name].columns)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [18]:
# Last 5 rows
rs = con.execute(f'''SELECT * 
                     FROM (SELECT *
                           FROM {table_name}
                           ORDER BY movieId DESC
                           LIMIT 5)
                     ORDER BY movieId ASC''')
pd.DataFrame(rs.fetchall(), columns=db[table_name].columns)

,movieId,imdbId,tmdbId
0,193581,5476944,432131.0
1,193583,5914996,445030.0
2,193585,6397426,479308.0
3,193587,8391976,483455.0
4,193609,101726,37891.0


> * **Pandas** 

In [19]:
# Top 5 rows
db[table_name].head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [20]:
# Last 5 rows
db[table_name].tail(5)

,movieId,imdbId,tmdbId
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0
9741,193609,101726,37891.0


### Query 2: Display Count of rows in a table

> * **SQL** 

In [0]:
table_name = 'ratings'

In [22]:
# all rows count
rs = con.execute(f'''SELECT COUNT(*) 
                    FROM {table_name}''')
pd.DataFrame(rs.fetchall(), columns=['count'])

,count
0,100836


In [23]:
# Distinct/Unique rows count
rs = con.execute(f'''SELECT DISTINCT COUNT(*) 
                     FROM {table_name}''')
pd.DataFrame(rs.fetchall(), columns=['distinct_count'])

,distinct_count
0,100836


> * **Pandas** 
    >> * There are muliple ways to display the row count in a table using pandas dataframe.
    >> 1. df.\__len__\()
    >> 2. df.shape[0]
    >> 3. len(df)
    >> 4. df.count()

In [24]:
# all rows count
db[table_name].__len__()

100836

In [25]:
# Distinct/Unique rows count
db[table_name].drop_duplicates().__len__()

100836

### Query 3: Display Count of Unique values in each column of a table

https://stackoverflow.com/questions/9889635/regular-expression-to-return-all-characters-between-two-special-characters

> * **SQL** 
> The sqlite_master table contains the following columns:

        Column_Name | Description        
        ----------- + -----------        
        type        | The type of database object such as table, index, trigger or view.
        name        | The name of the database object.
        tbl_name    | The table name that the database object is associated with.                     
        rootpage	| The Root page.
        sql         | SQL used to create the database object.

In [0]:
table_name = 'ratings'

In [0]:
def find_column_names_and_types(create_query):    
    create_query = re.sub('\n', '', create_query)
    create_query = re.sub('\t', '', create_query)
    match = re.match(r"^.*\((.*)\).*$", create_query)
    result = match.group(1)

    result = [r.strip('"').strip(',') for r in result.split()]    
    column_names = list(filter(lambda x: not x.isupper(), result))
    column_types = list(filter(lambda x: x.isupper(), result))
    return column_names, column_types

In [0]:
# Distinct/Unique rows count
def nunique(table_name):
    rs = con.execute(f"""SELECT sql AS create_query
                         FROM sqlite_master
                         WHERE tbl_name='{table_name}'""")
    create_query = rs.fetchall()[0][0]
    
    column_names, _ = find_column_names_and_types(create_query)

    result = []
    for column_name in column_names:
        rs = con.execute(f"""SELECT COUNT( DISTINCT {column_name})
                             FROM {table_name}
                             """)
        result.append((column_name, rs.fetchall()[0][0]))
    return result

In [29]:
result = nunique(table_name)
pd.DataFrame(result, columns=['column_name', 'unique_count'])

,column_name,unique_count
0,userId,610
1,movieId,9724
2,rating,10
3,timestamp,85043


> * **Pandas** 

In [30]:
pd.DataFrame(db[table_name].nunique(), columns = ['unique_count'])

,unique_count
userId,610
movieId,9724
rating,10
timestamp,85043


## `WHERE` Clause    
                  |  SQL  |  Pandas  |
    ----------------------------------              
       AND        |  and  |    &     |
    ----------------------------------              
       OR         |  or   |    |     |
    ----------------------------------              
       NOT        |  not  |    !     |

### Query 4: Find all the unique ratings given by a specific user with userId 25

> * **SQL** 

In [0]:
table_name = 'ratings'

In [32]:
rs = con.execute(f'''SELECT DISTINCT rating
                     FROM {table_name}
                     WHERE userId = 25''')
pd.DataFrame(rs.fetchall(), columns=['unique_ratings'])

,unique_ratings
0,4.0
1,5.0
2,4.5


> * **Pandas** 

In [33]:
table = db[table_name]

FROM_CLAUSE = db[table_name]

WHERE_CLAUSE = (table['userId'] == 25)

SELECT_CLAUSE = 'rating'

DISTINCT = pd.Series.unique

# 1. pd.Series.unique(db[table_name][(db[table_name]['userId'] == 25)]['rating'])
# 2. db[table_name][(db[table_name]['userId'] == 25)]['rating'].unique()

rs = DISTINCT(FROM_CLAUSE[WHERE_CLAUSE][SELECT_CLAUSE])
pd.DataFrame(rs,  columns=['unique_rating'])

,unique_rating
0,4.0
1,5.0
2,4.5


In [34]:
db[table_name][(db[table_name]['userId'] == 25)]['rating'].unique()

array([4. , 5. , 4.5])

### Query 5: Find all the movieId tagged as 'Leonardo DiCaprio' by user with userId 424

> * **SQL** 

In [0]:
table_name = 'tags'

In [36]:
rs = con.execute(f'''SELECT movieId
                     FROM {table_name}
                     WHERE tag = 'Leonardo DiCaprio' and userId = 424''')
pd.DataFrame(rs.fetchall(), columns=['movieId'])

,movieId
0,147
1,48516
2,74458
3,79132


> * **Pandas** 

In [37]:
table = db[table_name]

FROM_CLAUSE = table

WHERE_CONDITION1 = (table['tag'] == 'Leonardo DiCaprio')
WHERE_CONDITION2 = (table['userId'] == 424)
WHERE_CLAUSE = WHERE_CONDITION1 & WHERE_CONDITION2

SELECT_CLAUSE = ['movieId']

# db[table_name][(db[table_name]['tag'] == 'Leonardo DiCaprio') & \
#                (db[table_name]['userId'] == 424)] \
#                ['rating']

rs = FROM_CLAUSE[WHERE_CONDITION1 & WHERE_CONDITION2][SELECT_CLAUSE]
pd.DataFrame(rs)

,movieId
706,147
900,48516
923,74458
931,79132


### Query 6: Display all the movieId and tag in movies table for the movies tagged as music, romance and comedy.

> * **SQL** 

In [0]:
table_name = 'tags'

In [39]:
rs = con.execute(f'''SELECT movieId, tag
                     FROM {table_name}
                     WHERE tag IN ('music', 'comedy', 'romance')''')
pd.DataFrame(rs.fetchall(), columns=['movieId', 'tag'])

,movieId,tag
0,88094,music
1,46976,romance
2,60756,comedy
3,61024,comedy
4,88405,comedy
5,88405,romance
6,107348,comedy
7,119141,comedy
8,193565,comedy
9,1288,music


> * **Pandas** 

In [40]:
table = db[table_name]

FROM_CLAUSE = table

WHERE_CLAUSE = (table['tag'].isin(['music', 'comedy', 'romance']))

SELECT_CLAUSE = ['movieId', 'tag']

# db[table_name][db[table_name]['tag'].isin(['music', 'comedy', 'romance'])]

rs = FROM_CLAUSE[WHERE_CLAUSE][SELECT_CLAUSE]
pd.DataFrame(rs)

,movieId,tag
23,88094,music
178,46976,romance
205,60756,comedy
208,61024,comedy
238,88405,comedy
245,88405,romance
267,107348,comedy
304,119141,comedy
528,193565,comedy
622,1288,music


## `GROUP BY` Clause

### Query 7: Find the frequency distribution of movies based on ratings in descending order.


> * **SQL** 

In [0]:
table_name = 'ratings'

In [42]:
rs = con.execute(f'''SELECT rating, COUNT(movieId) AS count_movieId
                     FROM {table_name}
                     GROUP BY rating
                     ORDER BY count_movieId DESC''')
pd.DataFrame(rs.fetchall(), columns=['rating', 'movieId_count']).set_index('rating')

,movieId_count
rating,
4.0,26818
3.0,20047
5.0,13211
3.5,13136
4.5,8551
2.0,7551
2.5,5550
1.0,2811
1.5,1791


> * **Pandas** 

In [43]:
table = db[table_name]

FROM_CLAUSE = table

GROUPBY_COLUMN = ['rating']
GROUPBY_CLAUSE = pd.DataFrame.groupby

AGGREGATION_COLUMNS = ['movieId']
SELECT_CLAUSE = AGGREGATION_COLUMNS
AGGREGATION_OP = 'count'

# .sort_values() does the job of ORDER BY CLAUSE in SQL

# 1. pd.DataFrame.groupby( db[table_name], by=['rating'])[['movieId']].agg('count')
# 2. db[table_name].groupby(by=['rating'])[['movieId']].agg('count')
# 3. db[table_name].groupby(by=['rating']).agg({'movieId', 'count'})
# 4. db[table_name].groupby(by=['rating'])[['movieId']].count()


rs = GROUPBY_CLAUSE(FROM_CLAUSE, GROUPBY_COLUMN)[SELECT_CLAUSE] \
                                           .agg(AGGREGATION_OP) \
                                           .sort_values(by=['movieId'],
                                                        ascending=False)
pd.DataFrame(rs)

,movieId
rating,
4.0,26818
3.0,20047
5.0,13211
3.5,13136
4.5,8551
2.0,7551
2.5,5550
1.0,2811
1.5,1791


### Query 8: Find the number of tags assigned to each movie by each user.


> * **SQL** 

In [0]:
table_name = 'tags'

In [45]:
rs = con.execute(f'''SELECT userId, movieId, COUNT(tag)
                     FROM {table_name}
                     GROUP BY userId, movieId
                     ''')
pd.DataFrame(rs.fetchall(), columns=['userId', 'movieId', 'count_tag']).set_index(['userId', 'movieId'])

count_tag
userId movieId           
2      60756            3
       89774            3
       106782           3
7      48516            1
18     431              3
...                   ...
606    6107             1
       7382             1
       7936             1
610    3265             2
       168248           1

[1775 rows x 1 columns]

> * **Pandas** 

In [46]:
table = db[table_name]

FROM_CLAUSE = table

GROUPBY_COLUMN = ['userId', 'movieId']
GROUPBY_CLAUSE = pd.DataFrame.groupby

AGGREGATION_COLUMNS = ['tag']
SELECT_CLAUSE = AGGREGATION_COLUMNS
AGGREGATION_OP = 'count'

# Another way - db[table_name].groupby(by=['userId', 'movieId'])[['tag']].agg('count')

rs = GROUPBY_CLAUSE(FROM_CLAUSE, GROUPBY_COLUMN)[SELECT_CLAUSE] \
                                                .agg(AGGREGATION_OP)
                                                        
pd.DataFrame(rs)

tag
userId movieId     
2      60756      3
       89774      3
       106782     3
7      48516      1
18     431        3
...             ...
606    6107       1
       7382       1
       7936       1
610    3265       2
       168248     1

[1775 rows x 1 columns]

### Query 9: Find the maximum number of tags assigned to a movie by a user.


> * **SQL** 

In [0]:
table_name = 'tags'

In [48]:
rs = con.execute(f'''SELECT userId, movieId, MAX(count_tag)
                     FROM (SELECT userId, movieId, COUNT(tag) as count_tag
                           FROM {table_name}
                           GROUP BY userId, movieId) 
                     GROUP BY userId
                     ''')
pd.DataFrame(rs.fetchall(), columns=['userId', 'movieId', 'count_tag']).set_index(['userId', 'movieId']).T

userId,2,7,18,21,49,62,63,76,103,106,112,119,125,132,138,161,166,167,177,184,193,205,226,256,274,288,289,291,300,305,318,319,327,336,341,356,357,419,424,435,439,462,474,477,487,506,509,513,520,533,537,543,567,573,599,600,606,610
movieId,60756,48516,431,1569,109487,135536,260,260,296,4896,260,101142,7022,6367,59103,52287,54286,104,115617,2579,4878,260,6938,126548,68319,7020,3,50872,6711,4034,68954,364,1288,1,260,2146,91529,98961,3176,58559,98809,152711,7932,68791,112552,112552,80834,750,3039,356,79132,85565,7361,35836,296,273,1357,3265
count_tag,3,1,3,2,3,19,2,2,3,1,3,3,10,2,2,1,4,1,3,6,7,3,2,2,1,1,2,2,1,1,9,3,1,1,2,1,8,5,10,2,2,5,6,12,3,3,3,2,1,3,13,1,24,4,173,1,1,2


> * **Pandas** 

In [49]:
table = db[table_name]

FROM_CLAUSE = table

GROUPBY_COLUMNS1 = ['userId', 'movieId']
GROUPBY_COLUMNS2 = ['userId']
GROUPBY_CLAUSE = pd.DataFrame.groupby

SELECT_WITH_AGGREGATION_OP_DICT1 = {'tag': 'count'}
SELECT_WITH_AGGREGATION_OP_DICT2 = {'movieId': 'first', 'tag':'max'}

#  Another way - db[table_name].groupby(by=['userId', 'movieId'], \
#                                       as_index=False) \
#                              .agg({'tag': 'count'}) \
#                              .groupby(['userId'], as_index=True) \
#                              .agg({'movieId': 'first', 'tag':'max'})

# if we don't want to show the grouping columns as indexes of the dataframe, 
# we can put as_index=False in groupby function.

rs = GROUPBY_CLAUSE(GROUPBY_CLAUSE(FROM_CLAUSE, GROUPBY_COLUMNS1, 
                                   as_index=False)\
                                   .agg(SELECT_WITH_AGGREGATION_OP_DICT1), \
                    GROUPBY_COLUMNS2, as_index=True) \
                    .agg(SELECT_WITH_AGGREGATION_OP_DICT2)\
                                                    

                                                        
pd.DataFrame(rs).T

userId,2,7,18,21,49,62,63,76,103,106,112,119,125,132,138,161,166,167,177,184,193,205,226,256,274,288,289,291,300,305,318,319,327,336,341,356,357,419,424,435,439,462,474,477,487,506,509,513,520,533,537,543,567,573,599,600,606,610
movieId,60756,48516,431,1569,109487,2,260,260,260,4896,260,260,1726,3556,59103,52287,293,104,115617,2579,260,260,6938,126548,68319,7020,3,50872,6711,4034,778,364,1288,1,260,2146,39,98961,32,750,5952,152711,1,32,112552,112552,80834,750,3039,356,527,85565,1,248,293,273,1357,3265
tag,3,1,3,2,3,19,2,2,3,1,3,3,10,2,2,1,4,1,3,6,7,3,2,2,1,1,2,2,1,1,9,3,1,1,2,1,8,5,10,2,2,5,6,12,3,3,3,2,1,3,13,1,24,4,173,1,1,2
